# Iterable과 Iterator

- **Iterable**
    - 여러개의 데이터를 하나씩 또는 한 단위씩 제공하는 객체.
        - 다음 작업을 요청하면 값을 제공하며 for in 문에서 사용할 수 있다.
    - Iterator객체를 반환하는 `__iter__()` 특수 메소드를 정의해야 한다.
        - `__iter__()`는 `iter(Iterable)` 내장함수에 의해 호출된다. 
- **Iterator**
    - 자신을 생성한 Iterable의 값들을 하나씩 또는 한 단위씩 제공하는 객체
    - Iterable의 값을 제공하는 `__next__()` 특수 메소드를 정의한다.
        - `__next__()` 는 `next(Iterator)` 내정함수에 의해 호출된다.
        - 더 이상 제공할 값이 없을 경우 **StopIteration** Exception을 발생시켜야 한다.

In [1]:
l = [1, 2, 3]  # 리스트 > list --> iterable > __iter__() 구현
l_iterator = iter(l) # iter(iterable객체) -> iterable객체.__iter__()

print(type(l), type(l_iterator))

<class 'list'> <class 'list_iterator'>


In [2]:
# l.__iter__()

In [3]:
t = (1, 2, 3, 4)
t_iterator = iter(t)
print(type(t_iterator))

<class 'tuple_iterator'>


In [4]:
# iterator의 __next__() 메소드를 호출 --> next(iterator)
### 자기를 생성한 iterable객체의 원소를 하나 반환.
print(next(l_iterator))

1


In [5]:
print(next(l_iterator))

2


In [6]:
print(next(l_iterator))

3


In [7]:
print(next(l_iterator))

StopIteration: 

In [10]:
iterable = [1, 2, 3, 4, 5]
#1. iterable->iterator생성
iterator = iter(iterable)
while True:
    try:
        value = next(iterator)
        print(value)
    except StopIteration:
        break

print("다음작업")

1
2
3
4
5
다음작업


In [11]:
def forIn(iterable, func): 
    # iterable: 원소를 제공할 iterable 객체
    # func: 각 원소를 처리한 함수.
    iterator = iter(iterable)
    while True:
        try:
            value = next(iterator)
            func(value)
        except StopIteration:
            break
            
# for value in iterable:
#     xxx
#     xx

In [13]:
forIn([1,2, 3], lambda x : print(x))
forIn((10, 20, 30), lambda x : print(x))

1
2
3
10
20
30


## for in 문 Iterable의 값을 순환반복하는 과정

1. 반복 조회할 iterable객체의 __iter__() 를 호출 하여 Iterator를 구한다.
1. 매 반복마다 Iterator의 __next__() 를 호출하여 다음 원소를 조회한다.
1. 모든 원소들이 다 제공해 StopIteration Exception이 발생하면 반복문을 멈추고 빠져나온다.

In [37]:
# Iterable 과 Iterator를 구현
## 각각 다른 클래스로 구현
class MyIterable:
    
    def __init__(self, *args):
        # **args: 제공할 원소값들을 가변인자로 받기.
        self.values = args
    
    # iterable
    def __iter__(self):
        # Iterable은 반드시 __iter__() 특수 메소드를 구현해야한다.
        # __iter__()는 Iterator객체를 반환 하도록 구현.
        return MyIterator(self.values)
    
    # subscriptable - indexing 가능
    def __getitem__(self, index):
        return self.values[index]
    
    # len(객체) 호출되는 특수메소드 -> 원소의 개수를 반환.
    def __len__(self):
        return len(self.values)

In [38]:
i = MyIterable(1, 2, 3)
i[2]
len(i)

3

In [22]:
class MyIterator:
    
    def __init__(self, values):
        # Itarator가 제공할 값을 MyIterable로 부터 받는다.
        self.values = values
        self.index = 0  # 제공할 값의 index
        
    def __next__(self):
        # Iterator는 반드시 __next__() 특수 메소드를 구현해야한다.
        ## Iterable에서 받은 원소들에서 하나의 값을 순서대로 제공하도록 구현.
        ## 더 제공할 값이 없으면 StopIteration 예외를 발생시킨다.
        if len(self.values) <= self.index:
            raise StopIteration()
        r_value = self.values[self.index] # 제공할 값을 조회
        self.index += 1  #  index를 하나 증가
        return r_value

In [23]:
i = MyIterable(1, 2, 3, 4, 5) #iterable 생성
it = iter(i)  # iterable로 부터 iterator를 생성

In [24]:
# iterator로부터 한개의 값을 조회
next(it)

1

In [28]:
next(it)

5

In [30]:
it.index
next(it)

StopIteration: 

In [31]:
i = MyIterable(1, 2, 3, 4, 5) 
# for v in i:
for v in MyIterable(1, 2, 3, 4, 5):
    print(v)

1
2
3
4
5


In [32]:
i = MyIterable(1, 2, 3, 4, 5) 
# i[0] ==> Iterable에 def __getitem__(self, idx): idx번째 값을 반환하는 메소드

TypeError: 'MyIterable' object is not subscriptable

## Generator
- Iterable과 Iterator를 합친 기능을 함수 형태로 구현(정의)한 것을 generator라고 한다.
    - 제공할 값들을 미리 메모리에 올리지 않고 로직을 통해 값들을 호출자가 필요할 때 마다 제공할 때 유용하다.
- 제너레이터 함수에서 값을 반환
    - **yield 반환값**
        - 반환값을 가지고 호출한 곳으로 돌아간다. 현재 상태(돌아가기 직전 상태)를 기억하면서 돌아간다. 
            - 값을 반환하고 일시정지 상태라고 생각하면 된다.
        - 다음 실행시점에 yield 구문 다음 부터 실행된다.
    - **return \[valuye\]**
        - generator 함수 종료
        - StopIteration 발생시킨다.
- Generator 의 원소 조회
    - next(Generator객체)

In [42]:
def my_gen():
    
    yield 10

    yield 2

    yield 3

In [44]:
g = my_gen()  # generator  객체를 생성
# generator 호출
next(g)  # yield다음구문 ~ yield

10

In [45]:
next(g)

2

In [46]:
next(g)

3

In [47]:
next(g)

StopIteration: 

In [ ]:
v = next(g)
print(v)
print(v+10)
next(g)
next(g)
next(g)

In [48]:
# 파라미터로 받은 값에서 5씩 증가하는 값을 3번 제공.
def my_gen2(start):
    start += 5
    yield start
    
    start += 5
    yield start
    
    start += 5
    yield start

In [50]:
g2 = my_gen2(10)
print(next(g2))

15


In [51]:
print(next(g2))

20


In [52]:
print(next(g2))

25


In [53]:
print(next(g2))

StopIteration: 

In [58]:
def my_gen3(start, n):
    # start부터 5씩 증가하는 값을 n개 제공
    for _ in range(n):
        start += 5
        yield start

In [73]:
g3  = my_gen3(1, 10) # generator 객체 생성

In [74]:
v = next(g3)
print(v)
v = next(g3)
print(v)

6
11


### Generator 표현식 (Generator Comprehension)
- 컴프리헨션구문을 **( )** 로 묶어 표현한다.
- 컴프리헨션 구문안의 Iterable의 원소들을 처리해서 제공하는 generator 표현식
- Generator Comprehension 은 반복 가능한 객체만 만들고 실제 원소에 대한 요청이 왔을 때 값을 생성한다.
    - 메모리 효율이 다른 Comprehension들 보다 좋다.

In [84]:
g4 = (value+5 for value in range(10) )

In [91]:
next(g4)

11

In [ ]:
l = [1, 2, 3, 4]
(v for v in range(1, 5))

# Decorator (장식자)

## 파이썬에서 함수는 일급 시민(first class citizen) 이다.
- 일급 시민 (first class citizen) 이란
    1. 변수에 대입 할 수 있다.
    2. Argument로 사용할 수 있다.
    3. 함수나 메소드의 반환값으로 사용 할 수 있다.
    

## 지역함수(Local Function) 란
- 함수 안에 정의 한 함수를 말한다.
    - 중첩 함수(Nested function) 이라고도 한다.
- 지역함수가 선언된 함수를 **outer function** 지역함수는 **inner function** 이라고 한다. 
- inner function은 outer function의 지역변수를 자유롭게 사용할 수 있다.
- 기본적으로 inner function은 outer function 안에서만 호출 할 수있다.
- 단 outer function이 정의된 inner function을 return value로 반환하면 밖에서도 호출 할 수 있다.

## Closure (클로저)
- 지역함수(Inner function)를 정의한 Outer function이 종료되어도 지역함수가 종료될 때까지 outer function의 지역변수들은 메모리에 계속 유지 되어 inner function에서 사용할 수 있다. 
- 파이썬 실행환경은 inner function이 종료될때 까지 outer function의 지역변수들(parameter포함)을 사용할 수 있도록 저장하는 공간이 **closure**이다.

## Decorator (장식자)
- 기존의 함수를 수정하지 않고 그 함수 전/후에 실행되는 구문을 추가할 수 있도록 하는 함수를 말한다.
- 기존 함수코드를 수정하지 않고 새로운 기능의 추가를 쉽게 해준다.
- 추가기능을 다수의 함수에 적용할 수 있다.
- 함수의 전/후처리 하는 구문을 **필요하면 붙이고 필요 없으면 쉽게 제거할 수 있다**

![개요](images/ch10_01.png)

### Decorator 구현 및 사용

- 구현
    1. 전/후처리 기능을 추가할 함수를 parameter로 받는다.
    2. 그 함수 호출 전후로 추가할 기능을 작성한 **지역함수**를 정의한다.
    3. `2`번의 함수를 반환한다.
```python
def decorator(func):
    def wrapper([parameter]): # decorator 적용할 함수에 파라미터를 전달할 경우 parameter 변수들을 선언
        # 전처리
        func()
        # 후처리
    return wrapper 
```

- 호출
    - `@decorator이름`를 적용하고자하는 함수 선언전에 기술한다.
```python
@decorator
def caller([parameter]):
    ...
```

# TODO
함수가 실행된 실행시간(초)을 재는 decorator